# AWS Sage maker Introduction
* ファイルの読み書きはすべてS3上で実施
* 参考：　https://recipe.kc-cloud.jp/archives/10921
* https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb

## 共通実施事項
* sagemakerライブラリのインポート
* S3情報の登録
* IAMアカウント情報の登録

IAMのroleの宣言

In [138]:
import boto3
import re
import sagemaker
from sagemaker import get_execution_role
import pandas as pd
import io
import os
role = get_execution_role()

## S3
### 登録
読み込みと書き込みで用いる変数の指定

In [139]:
BUCKET = 'yourbucketname' #バケット名を入力
inpath = "file/to/path/" #入力データまでのパス
infile = "scotlip.csv" #入力データのファイル名
KEY = inpath + infile

outpath = "out/to/path/"
outfile = "scot_test.csv"
outKEY = outpath + outfile

### 読み込み

In [140]:
# S3からCSVをダウンロード
s3 = boto3.client('s3')
response = s3.get_object(Bucket=BUCKET, Key=KEY)

data = pd.read_csv(io.StringIO(response['Body'].read().decode('utf-8'))) #decode以下で文字コードを指定する

In [141]:
data.head(5)

,CODENO,AREA,PERIMETER,RECORD_ID,DISTRICT,NAME,CODE,CANCER,POP,CEXP,AFF
0,6126,9.740020e+08,184951.0,1,1,Skye-Lochalsh,w6126,9,28324,1.38,16
1,6016,1.461990e+09,178224.0,2,2,Banff-Buchan,w6016,39,231337,8.66,16
2,6121,1.753090e+09,179177.0,3,3,Caithness,w6121,11,83190,3.04,10
3,5601,8.985990e+08,128777.0,4,4,Berwickshire,w5601,9,51710,2.53,24
4,6125,5.109870e+09,580792.0,5,5,Ross-Cromarty,w6125,15,129271,4.26,10


In [142]:
test = data["NAME"]
test.head(3)

0    Skye-Lochalsh
1     Banff-Buchan
2        Caithness
Name: NAME, dtype: object

### 書き込み
一旦SageMakerの/home上に書き込んだのち、botoを用いてS3に移動。
参考　https://qiita.com/is_ryo/items/e16527db5800854cd95f

In [143]:
test.to_csv(outfile)

In [144]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(BUCKET)
bucket.upload_file(outfile, outKEY)

### home/上のファイルを削除
容量節約のため、必ず行うこと。

In [148]:
os.remove(outfile)

### ライブラリのインストール
* 試験的にpyprojライブラリをインストール
* 参考：　[公式ガイド](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/nbi-add-external.html)

⇒問題なくインストール・インポートに成功　Colab上と同様。

In [124]:
!pip install pyproj

    100% |████████████████████████████████| 4.4MB 9.0MB/s eta 0:00:01
  Running setup.py bdist_wheel for pyproj ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/89/77/ec/a537585d1022dafde0317dd19d33c4a30d4ee61e19f25ebd8e
Successfully built pyproj
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [125]:
import pyproj